## This notebook contains detailed analysis of Titanic dataset and then predics the survival of test data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [ ]:
data=pd.read_csv("titanic/train.csv")
test_data=pd.read_csv("titanic/test.csv")

In [ ]:
data.isnull().sum()

<p>Dropping Cabin columns as it has multiple null values</p>
Also fill the null age as average age and null embarked with mode of embarked

In [ ]:
data.drop(columns=["Cabin"], inplace=True)
test_data.drop(columns=["Cabin"], inplace=True)

In [ ]:
#data["Age"].fillna(data["Age"].mean(),inplace=True)
data.fillna({"Age":data["Age"].mean(),"Embarked":data["Embarked"].mode()[0]},inplace=True)

In [ ]:
test_data.isnull().sum()

In [ ]:
#test_data["Age"].fillna(data["Age"].mean(),inplace=True)
test_data.fillna({"Age":test_data["Age"].mean(),"Fare":test_data["Fare"].mean()},inplace=True)

Now that we have cleaned data we are ready to prepare our data for ML modeling.

In [ ]:
data["Sex"]=data["Sex"].map({"male":0,"female":1})

In [ ]:
test_data["Sex"]=test_data["Sex"].map({"male":0,"female":1})

In [ ]:
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})
test_data["Embarked"] = test_data["Embarked"].map({"S": 0, "C": 1, "Q": 2})


In [ ]:
data["Family Size"]=data["Parch"]+data["SibSp"]
test_data["Family Size"]=test_data["Parch"]+test_data["SibSp"]

In [ ]:
data.head()

In [ ]:
test_data.head()

In [ ]:
features = ["Pclass", "Sex", "Age", "Fare", "Embarked", "Family Size"]
x_train = data[features]
y_train = data["Survived"]
x_test = test_data[features]


In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)

In [ ]:
pipeline = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(degree=2, include_bias=False),
    LogisticRegression(max_iter=1000, solver='saga')
)


In [ ]:
pipeline.fit(x_train, y_train)
scores = cross_val_score(pipeline, x_train, y_train, cv=5)
print("CV Accuracy:", scores.mean())

In [ ]:
y_pred = pipeline.predict(x_test)

In [ ]:
print(y_pred)

In [ ]:
# Save results
result = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": y_pred
})
result.to_csv('submission.csv', index=False)
